In [98]:
table_a = pd.DataFrame({
    'A': ['all', 1, 4, 4,6,5,8,'all'], 
    'B': [2, 2, 3, 2,6,7,12,'all'], 
    'C': [3, 3, 3, 3,6,1,8,13], 
    'D': [4, 4, 4, 4,6,1,13,9]
})

table_b = pd.DataFrame({
    'A': ['all', 1, 'all', 4,5,'all','all'], 
    'B': [2, 2, 2, 'all',7,'all','all'], 
    'C': [3, 3, 'all', 3,'all',8,13], 
    'target': ['xyz', 'zyx', 'tyz', 'abc','yyy','ppp','hgf']
})

In [99]:
table_a


,A,B,C,D
0,all,2,3,4
1,1,2,3,4
2,4,3,3,4
3,4,2,3,4
4,6,6,6,6
5,5,7,1,1
6,8,12,8,13
7,all,all,13,9


In [100]:
table_b

,A,B,C,target
0,all,2,3,xyz
1,1,2,3,zyx
2,all,2,all,tyz
3,4,all,3,abc
4,5,7,all,yyy
5,all,all,8,ppp
6,all,all,13,hgf


In [101]:
import pandas as pd
import numpy as np

def get_label(table_a, table_b, priority_map):
    # Determine common columns between both DataFrames, excluding 'target'
    common_columns = list(set(table_a.columns).intersection(set(table_b.columns)) - {'target'})

    # Nested function for finding the best match based on provided conditions
    def find_best_match(row):
        conditions = []  # List to hold conditions for matching
        
        # Iterate through common columns
        for col in common_columns:
            # Append conditions for each column. 
            # The condition checks if value in Table B is same as in Table A or it is 'all' or null
            conditions.append(
                (table_b[col]==row[col]) | (table_b[col]=='all') | (table_b[col].isnull()))
        
        # Combine all conditions to find matching rows in table_b
        matching_rows = table_b[np.logical_and.reduce(conditions)]

        # If matching rows are found
        if not matching_rows.empty:
            # Assign scores based on priority_map
            # Scores are calculated by checking if values are not None and not 'all', and then multiplying by priority
            scores = matching_rows[common_columns].applymap(lambda x: x is not None and x != 'all').dot(
                [priority_map[col] for col in common_columns])
            # Return the target value of the row with the highest score
            return matching_rows.loc[scores.idxmax()]['target']

        # If no matching rows are found, return None
        return None

    # Apply the find_best_match function to each row in table_a to find the corresponding 'target' value
    table_a['target'] = table_a.apply(find_best_match, axis=1)
    # Return the updated table_a as the final output
    return table_a




In [102]:
priority_map ={'A' : 3, 'B' : 2, 'C':1}
table_a = pd.DataFrame({'A': ['all', 1, 4, 4,5], 
                        'B': [2, 2, 3, 2,2],
                        'C': [3, 3, 3, 3,7],
                        'D': [4, 4, 4, 4,9]})

table_b = pd.DataFrame({'A': ['all', 1, 'all'],
        'B': [2, 2, 2],
        'C': [3, 3, 'all'],
        'target': ['xyz', 'zyx', 'tyz']})

get_label(table_a, table_b, priority_map)

,A,B,C,D,target
0,all,2,3,4,xyz
1,1,2,3,4,zyx
2,4,3,3,4,None
3,4,2,3,4,xyz
4,5,2,7,9,tyz


In [103]:
priority_map ={'A' : 3, 'B' : 2, 'C':1}

table_a = pd.DataFrame({
    'A': ['all', 1, 4, 4,6,5,8,'all'], 
    'B': [2, 2, 3, 2,6,7,12,'all'], 
    'C': [3, 3, 3, 3,6,1,8,13], 
    'D': [4, 4, 4, 4,6,1,13,9]
})

table_b = pd.DataFrame({
    'A': ['all', 1, 'all', 4,5,'all','all'], 
    'B': [2, 2, 2, 'all',7,'all','all'], 
    'C': [3, 3, 'all', 3,'all',8,13], 
    'target': ['xyz', 'zyx', 'tyz', 'abc','yyy','ppp','hgf']
})

get_label(table_a, table_b, priority_map)

,A,B,C,D,target
0,all,2,3,4,xyz
1,1,2,3,4,zyx
2,4,3,3,4,abc
3,4,2,3,4,abc
4,6,6,6,6,None
5,5,7,1,1,yyy
6,8,12,8,13,ppp
7,all,all,13,9,hgf
